# 03 - Model Explainability & Fairness Analysis

This notebook provides comprehensive model explainability:
- SHAP values computation for tree-based models
- Summary and force plots
- Per-segment performance analysis
- Automated fairness checks (demographic parity, equalized odds)

## 1. Setup & Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# ML imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix
)
import xgboost as xgb
import lightgbm as lgb

# SHAP for explainability
import shap

# MLflow
import mlflow

# Visualization settings
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

# Initialize SHAP JS visualization
shap.initjs()

print("Libraries loaded successfully!")
print(f"SHAP version: {shap.__version__}")

## 2. Configuration

In [ ]:
# Project paths
PROJECT_ROOT = Path(".").resolve().parent
DATA_PATH = PROJECT_ROOT / "data" / "sample" / "events.parquet"
OUTPUT_DIR = Path(".")

# MLflow configuration
MLFLOW_TRACKING_URI = PROJECT_ROOT / "mlruns"
mlflow.set_tracking_uri(str(MLFLOW_TRACKING_URI))

print(f"Project root: {PROJECT_ROOT}")
print(f"Data path: {DATA_PATH}")

## 3. Data Loading & Preparation

In [ ]:
def load_and_prepare_data(data_path: Path) -> pd.DataFrame:
    """Load and prepare events data."""
    df = pd.read_parquet(data_path)
    print(f"Loaded {len(df):,} events")
    
    # Parse metadata
    if 'metadata' in df.columns and df['metadata'].dtype == 'object':
        df['metadata'] = df['metadata'].apply(
            lambda x: json.loads(x) if isinstance(x, str) else x
        )
    
    # Extract features
    df['channel'] = df['metadata'].apply(lambda x: x.get('channel') if isinstance(x, dict) else 'unknown')
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['revenue'] = df['price'] * df['quantity']
    
    # Extract region from location
    if 'location' in df.columns:
        df['region'] = df['location'].apply(
            lambda x: x.split(',')[-1].strip() if isinstance(x, str) and ',' in x else 'Unknown'
        )
    else:
        df['region'] = 'Unknown'
    
    return df


df = load_and_prepare_data(DATA_PATH)
print(f"\nDataset shape: {df.shape}")
print(f"Unique regions: {df['region'].nunique()}")

In [ ]:
def create_user_features(df: pd.DataFrame) -> pd.DataFrame:
    """Create user-level features with cohort segmentation."""
    max_date = df['timestamp'].max()
    
    user_features = df.groupby('user_id').agg({
        'id': 'count',
        'revenue': ['sum', 'mean', 'std'],
        'price': ['mean', 'max', 'min'],
        'quantity': ['sum', 'mean'],
        'timestamp': ['min', 'max'],
        'channel': lambda x: x.mode().iloc[0] if len(x) > 0 else 'unknown',
        'region': lambda x: x.mode().iloc[0] if len(x) > 0 else 'Unknown'
    }).reset_index()
    
    # Flatten column names
    user_features.columns = [
        'user_id', 'transaction_count', 'total_revenue', 'avg_revenue', 'std_revenue',
        'avg_price', 'max_price', 'min_price', 'total_quantity', 'avg_quantity',
        'first_transaction', 'last_transaction', 'primary_channel', 'primary_region'
    ]
    
    # Derived features
    user_features['days_since_first'] = (max_date - user_features['first_transaction']).dt.days
    user_features['days_since_last'] = (max_date - user_features['last_transaction']).dt.days
    user_features['avg_days_between'] = user_features['days_since_first'] / user_features['transaction_count'].clip(lower=1)
    user_features['std_revenue'] = user_features['std_revenue'].fillna(0)
    
    # Create user cohorts based on total revenue
    user_features['user_cohort'] = pd.qcut(
        user_features['total_revenue'],
        q=3,
        labels=['Low Value', 'Medium Value', 'High Value']
    )
    
    # Create churn label (synthetic based on recency and revenue)
    revenue_threshold = user_features['total_revenue'].quantile(0.35)
    recency_threshold = user_features['days_since_last'].quantile(0.65)
    
    user_features['churned'] = (
        (user_features['total_revenue'] < revenue_threshold) |
        ((user_features['total_revenue'] < user_features['total_revenue'].quantile(0.5)) &
         (user_features['days_since_last'] > recency_threshold))
    ).astype(int)
    
    # Drop timestamp columns
    user_features = user_features.drop(['first_transaction', 'last_transaction'], axis=1)
    
    return user_features


user_df = create_user_features(df)
print(f"User dataset: {user_df.shape}")
print(f"\nChurn rate: {user_df['churned'].mean():.2%}")
print(f"\nUser cohort distribution:")
print(user_df['user_cohort'].value_counts())

## 4. Train Tree Models

In [ ]:
# Define features
feature_cols = [
    'transaction_count', 'total_revenue', 'avg_revenue', 'std_revenue',
    'avg_price', 'max_price', 'min_price', 'total_quantity', 'avg_quantity',
    'days_since_first', 'days_since_last', 'avg_days_between'
]

X = user_df[feature_cols]
y = user_df['churned']

# Keep metadata for segment analysis
segment_info = user_df[['user_id', 'primary_region', 'user_cohort', 'primary_channel']].copy()

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Split segment info accordingly
segment_test = segment_info.iloc[X_test.index].reset_index(drop=True)
X_test_reset = X_test.reset_index(drop=True)
y_test_reset = y_test.reset_index(drop=True)

print(f"Training set: {X_train.shape[0]:,} samples")
print(f"Test set: {X_test.shape[0]:,} samples")

In [ ]:
# Train XGBoost model
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    random_state=42,
    eval_metric='logloss'
)
xgb_model.fit(X_train, y_train)

# Train LightGBM model
lgb_model = lgb.LGBMClassifier(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    random_state=42,
    verbose=-1
)
lgb_model.fit(X_train, y_train)

# Get predictions
xgb_pred = xgb_model.predict(X_test_reset)
lgb_pred = lgb_model.predict(X_test_reset)

print("XGBoost Metrics:")
print(f"  Accuracy: {accuracy_score(y_test_reset, xgb_pred):.4f}")
print(f"  AUC-ROC: {roc_auc_score(y_test_reset, xgb_model.predict_proba(X_test_reset)[:, 1]):.4f}")

print("\nLightGBM Metrics:")
print(f"  Accuracy: {accuracy_score(y_test_reset, lgb_pred):.4f}")
print(f"  AUC-ROC: {roc_auc_score(y_test_reset, lgb_model.predict_proba(X_test_reset)[:, 1]):.4f}")

## 5. SHAP Values Computation

In [ ]:
# Compute SHAP values for XGBoost
print("Computing SHAP values for XGBoost...")
xgb_explainer = shap.TreeExplainer(xgb_model)
xgb_shap_values = xgb_explainer.shap_values(X_test_reset)

print("Computing SHAP values for LightGBM...")
lgb_explainer = shap.TreeExplainer(lgb_model)
lgb_shap_values = lgb_explainer.shap_values(X_test_reset)

# For binary classification, LightGBM returns list of [class_0, class_1]
if isinstance(lgb_shap_values, list):
    lgb_shap_values = lgb_shap_values[1]  # Take positive class

print(f"\nXGBoost SHAP values shape: {xgb_shap_values.shape}")
print(f"LightGBM SHAP values shape: {lgb_shap_values.shape}")

## 6. SHAP Summary Plots

In [ ]:
# XGBoost Summary Plot (Beeswarm)
print("XGBoost SHAP Summary Plot")
plt.figure(figsize=(12, 8))
shap.summary_plot(xgb_shap_values, X_test_reset, feature_names=feature_cols, show=False)
plt.title('XGBoost - SHAP Feature Importance', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('shap_summary_xgboost.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# LightGBM Summary Plot (Beeswarm)
print("LightGBM SHAP Summary Plot")
plt.figure(figsize=(12, 8))
shap.summary_plot(lgb_shap_values, X_test_reset, feature_names=feature_cols, show=False)
plt.title('LightGBM - SHAP Feature Importance', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('shap_summary_lightgbm.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Bar plot of mean absolute SHAP values
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# XGBoost
shap.summary_plot(xgb_shap_values, X_test_reset, feature_names=feature_cols, 
                  plot_type='bar', show=False, ax=axes[0])
axes[0].set_title('XGBoost - Mean |SHAP|', fontsize=12, fontweight='bold')

# LightGBM
shap.summary_plot(lgb_shap_values, X_test_reset, feature_names=feature_cols,
                  plot_type='bar', show=False, ax=axes[1])
axes[1].set_title('LightGBM - Mean |SHAP|', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.savefig('shap_bar_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. SHAP Force Plots

In [ ]:
# Force plot for a single prediction (XGBoost)
sample_idx = 0
print(f"Force plot for sample {sample_idx}:")
print(f"  Actual: {'Churned' if y_test_reset.iloc[sample_idx] == 1 else 'Not Churned'}")
print(f"  Predicted: {'Churned' if xgb_pred[sample_idx] == 1 else 'Not Churned'}")
print(f"  Probability: {xgb_model.predict_proba(X_test_reset)[sample_idx, 1]:.3f}")

shap.force_plot(
    xgb_explainer.expected_value,
    xgb_shap_values[sample_idx],
    X_test_reset.iloc[sample_idx],
    feature_names=feature_cols,
    matplotlib=True,
    show=False
)
plt.title('XGBoost Force Plot - Single Prediction', fontsize=12)
plt.tight_layout()
plt.savefig('shap_force_plot_single.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Force plot for multiple predictions (interactive in notebook)
print("Interactive force plot for first 100 predictions:")
shap.force_plot(
    xgb_explainer.expected_value,
    xgb_shap_values[:100],
    X_test_reset.iloc[:100],
    feature_names=feature_cols
)

In [ ]:
# Dependence plot for top feature
top_feature = 'total_revenue'
print(f"\nDependence plot for '{top_feature}':")

plt.figure(figsize=(10, 6))
shap.dependence_plot(
    top_feature,
    xgb_shap_values,
    X_test_reset,
    feature_names=feature_cols,
    show=False
)
plt.title(f'SHAP Dependence Plot - {top_feature}', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.savefig('shap_dependence_total_revenue.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Per-Segment Performance Analysis

In [ ]:
def compute_segment_metrics(y_true, y_pred, y_prob, segment_labels):
    """Compute metrics per segment."""
    results = []
    
    for segment in segment_labels.unique():
        mask = segment_labels == segment
        n_samples = mask.sum()
        
        if n_samples < 5:  # Skip segments with too few samples
            continue
        
        segment_y_true = y_true[mask]
        segment_y_pred = y_pred[mask]
        segment_y_prob = y_prob[mask]
        
        # Handle edge case where segment has only one class
        if len(segment_y_true.unique()) < 2:
            auc = np.nan
        else:
            auc = roc_auc_score(segment_y_true, segment_y_prob)
        
        results.append({
            'segment': segment,
            'n_samples': n_samples,
            'churn_rate': segment_y_true.mean(),
            'accuracy': accuracy_score(segment_y_true, segment_y_pred),
            'precision': precision_score(segment_y_true, segment_y_pred, zero_division=0),
            'recall': recall_score(segment_y_true, segment_y_pred, zero_division=0),
            'f1': f1_score(segment_y_true, segment_y_pred, zero_division=0),
            'auc_roc': auc
        })
    
    return pd.DataFrame(results)


# Get predictions and probabilities
xgb_prob = xgb_model.predict_proba(X_test_reset)[:, 1]

In [ ]:
# Performance by Region
print("=" * 60)
print("PERFORMANCE BY REGION")
print("=" * 60)

region_metrics = compute_segment_metrics(
    y_test_reset,
    xgb_pred,
    xgb_prob,
    segment_test['primary_region']
)

print(region_metrics.to_string(index=False))

# Visualize
if len(region_metrics) > 0:
    fig, ax = plt.subplots(figsize=(12, 6))
    x = np.arange(len(region_metrics))
    width = 0.2
    
    ax.bar(x - width*1.5, region_metrics['accuracy'], width, label='Accuracy', color='steelblue')
    ax.bar(x - width/2, region_metrics['precision'], width, label='Precision', color='coral')
    ax.bar(x + width/2, region_metrics['recall'], width, label='Recall', color='seagreen')
    ax.bar(x + width*1.5, region_metrics['f1'], width, label='F1', color='gold')
    
    ax.set_xticks(x)
    ax.set_xticklabels(region_metrics['segment'], rotation=45, ha='right')
    ax.set_ylabel('Score')
    ax.set_title('Model Performance by Region', fontsize=12, fontweight='bold')
    ax.legend()
    ax.set_ylim(0, 1.1)
    
    plt.tight_layout()
    plt.savefig('segment_performance_region.png', dpi=150, bbox_inches='tight')
    plt.show()

In [ ]:
# Performance by User Cohort
print("\n" + "=" * 60)
print("PERFORMANCE BY USER COHORT")
print("=" * 60)

cohort_metrics = compute_segment_metrics(
    y_test_reset,
    xgb_pred,
    xgb_prob,
    segment_test['user_cohort'].astype(str)
)

print(cohort_metrics.to_string(index=False))

# Visualize
if len(cohort_metrics) > 0:
    fig, ax = plt.subplots(figsize=(10, 6))
    x = np.arange(len(cohort_metrics))
    width = 0.2
    
    ax.bar(x - width*1.5, cohort_metrics['accuracy'], width, label='Accuracy', color='steelblue')
    ax.bar(x - width/2, cohort_metrics['precision'], width, label='Precision', color='coral')
    ax.bar(x + width/2, cohort_metrics['recall'], width, label='Recall', color='seagreen')
    ax.bar(x + width*1.5, cohort_metrics['f1'], width, label='F1', color='gold')
    
    ax.set_xticks(x)
    ax.set_xticklabels(cohort_metrics['segment'])
    ax.set_ylabel('Score')
    ax.set_title('Model Performance by User Cohort', fontsize=12, fontweight='bold')
    ax.legend()
    ax.set_ylim(0, 1.1)
    
    plt.tight_layout()
    plt.savefig('segment_performance_cohort.png', dpi=150, bbox_inches='tight')
    plt.show()

In [ ]:
# Performance by Channel
print("\n" + "=" * 60)
print("PERFORMANCE BY CHANNEL")
print("=" * 60)

channel_metrics = compute_segment_metrics(
    y_test_reset,
    xgb_pred,
    xgb_prob,
    segment_test['primary_channel']
)

print(channel_metrics.to_string(index=False))

## 9. Automated Fairness Checks

In [ ]:
def compute_demographic_parity(y_pred, sensitive_feature):
    """
    Compute demographic parity (statistical parity).
    
    Demographic parity is satisfied when the selection rate (positive prediction rate)
    is equal across all groups defined by the sensitive feature.
    
    Returns: DataFrame with selection rates per group and parity ratio.
    """
    results = []
    
    for group in sensitive_feature.unique():
        mask = sensitive_feature == group
        n_samples = mask.sum()
        selection_rate = y_pred[mask].mean()
        
        results.append({
            'group': group,
            'n_samples': n_samples,
            'selection_rate': selection_rate
        })
    
    df = pd.DataFrame(results)
    
    # Compute parity ratio (min/max selection rate)
    min_rate = df['selection_rate'].min()
    max_rate = df['selection_rate'].max()
    parity_ratio = min_rate / max_rate if max_rate > 0 else 1.0
    
    return df, parity_ratio


def compute_equalized_odds(y_true, y_pred, sensitive_feature):
    """
    Compute equalized odds metrics.
    
    Equalized odds is satisfied when both TPR and FPR are equal across groups.
    
    Returns: DataFrame with TPR and FPR per group.
    """
    results = []
    
    for group in sensitive_feature.unique():
        mask = sensitive_feature == group
        group_y_true = y_true[mask]
        group_y_pred = y_pred[mask]
        
        # True Positive Rate (TPR) = Recall
        positives = group_y_true == 1
        if positives.sum() > 0:
            tpr = (group_y_pred[positives] == 1).mean()
        else:
            tpr = np.nan
        
        # False Positive Rate (FPR)
        negatives = group_y_true == 0
        if negatives.sum() > 0:
            fpr = (group_y_pred[negatives] == 1).mean()
        else:
            fpr = np.nan
        
        results.append({
            'group': group,
            'n_samples': mask.sum(),
            'tpr': tpr,
            'fpr': fpr
        })
    
    df = pd.DataFrame(results)
    
    # Compute equalized odds gap (max difference in TPR and FPR)
    tpr_gap = df['tpr'].max() - df['tpr'].min()
    fpr_gap = df['fpr'].max() - df['fpr'].min()
    
    return df, tpr_gap, fpr_gap

In [ ]:
# Fairness Analysis by User Cohort
print("=" * 60)
print("FAIRNESS ANALYSIS - BY USER COHORT")
print("=" * 60)

# Demographic Parity
dp_df, dp_ratio = compute_demographic_parity(
    xgb_pred,
    segment_test['user_cohort'].astype(str)
)

print("\n--- Demographic Parity ---")
print(dp_df.to_string(index=False))
print(f"\nParity Ratio (min/max selection rate): {dp_ratio:.3f}")
print(f"{'✓ PASS' if dp_ratio >= 0.8 else '✗ FAIL'}: Threshold is 0.80 (80% rule)")

# Equalized Odds
eo_df, tpr_gap, fpr_gap = compute_equalized_odds(
    y_test_reset.values,
    xgb_pred,
    segment_test['user_cohort'].astype(str)
)

print("\n--- Equalized Odds ---")
print(eo_df.to_string(index=False))
print(f"\nTPR Gap (max - min): {tpr_gap:.3f}")
print(f"FPR Gap (max - min): {fpr_gap:.3f}")
print(f"{'✓ PASS' if tpr_gap <= 0.1 and fpr_gap <= 0.1 else '✗ REVIEW'}: TPR/FPR gap threshold is 0.10")

In [ ]:
# Fairness Analysis by Region
print("\n" + "=" * 60)
print("FAIRNESS ANALYSIS - BY REGION")
print("=" * 60)

# Demographic Parity
dp_region_df, dp_region_ratio = compute_demographic_parity(
    xgb_pred,
    segment_test['primary_region']
)

print("\n--- Demographic Parity ---")
print(dp_region_df.to_string(index=False))
print(f"\nParity Ratio: {dp_region_ratio:.3f}")
print(f"{'✓ PASS' if dp_region_ratio >= 0.8 else '✗ FAIL'}: Threshold is 0.80")

# Equalized Odds
eo_region_df, tpr_region_gap, fpr_region_gap = compute_equalized_odds(
    y_test_reset.values,
    xgb_pred,
    segment_test['primary_region']
)

print("\n--- Equalized Odds ---")
print(eo_region_df.to_string(index=False))
print(f"\nTPR Gap: {tpr_region_gap:.3f}")
print(f"FPR Gap: {fpr_region_gap:.3f}")

In [ ]:
# Fairness Summary Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Demographic Parity by Cohort
ax1 = axes[0]
colors = ['coral' if r < 0.8 else 'seagreen' for r in [dp_ratio]]
bars = ax1.bar(dp_df['group'], dp_df['selection_rate'], color='steelblue', edgecolor='black')
ax1.axhline(y=dp_df['selection_rate'].mean(), color='red', linestyle='--', label='Mean Rate')
ax1.set_ylabel('Selection Rate (Churn Prediction)')
ax1.set_xlabel('User Cohort')
ax1.set_title(f'Demographic Parity by Cohort\n(Parity Ratio: {dp_ratio:.2f})', fontsize=12, fontweight='bold')
ax1.legend()
ax1.set_ylim(0, 1)

# Equalized Odds by Cohort
ax2 = axes[1]
x = np.arange(len(eo_df))
width = 0.35
ax2.bar(x - width/2, eo_df['tpr'], width, label='TPR', color='steelblue')
ax2.bar(x + width/2, eo_df['fpr'], width, label='FPR', color='coral')
ax2.set_xticks(x)
ax2.set_xticklabels(eo_df['group'])
ax2.set_ylabel('Rate')
ax2.set_xlabel('User Cohort')
ax2.set_title(f'Equalized Odds by Cohort\n(TPR Gap: {tpr_gap:.2f}, FPR Gap: {fpr_gap:.2f})', fontsize=12, fontweight='bold')
ax2.legend()
ax2.set_ylim(0, 1)

plt.tight_layout()
plt.savefig('fairness_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

## 10. Export SHAP Data for API

In [ ]:
def export_shap_summary(shap_values, X, feature_names, output_path):
    """Export SHAP summary data as JSON for API consumption."""
    
    # Compute mean absolute SHAP values
    mean_abs_shap = np.abs(shap_values).mean(axis=0)
    
    # Create summary
    summary = {
        'feature_importance': [
            {'feature': f, 'mean_abs_shap': float(v)}
            for f, v in sorted(zip(feature_names, mean_abs_shap), key=lambda x: -x[1])
        ],
        'expected_value': float(xgb_explainer.expected_value),
        'n_samples': len(X),
        'model_type': 'XGBoost',
        'generated_at': datetime.now().isoformat()
    }
    
    with open(output_path, 'w') as f:
        json.dump(summary, f, indent=2)
    
    print(f"SHAP summary exported to: {output_path}")
    return summary


# Export global summary
shap_summary = export_shap_summary(
    xgb_shap_values,
    X_test_reset,
    feature_cols,
    'shap_summary.json'
)

print("\nTop 5 Features by SHAP Importance:")
for item in shap_summary['feature_importance'][:5]:
    print(f"  {item['feature']}: {item['mean_abs_shap']:.4f}")

In [ ]:
def export_individual_explanation(idx, shap_values, X, feature_names, explainer, y_pred, y_prob):
    """Export individual prediction explanation."""
    
    explanation = {
        'prediction_id': f'pred_{idx}',
        'predicted_class': int(y_pred[idx]),
        'predicted_probability': float(y_prob[idx]),
        'base_value': float(explainer.expected_value),
        'shap_values': [
            {'feature': f, 'value': float(X.iloc[idx][f]), 'shap': float(s)}
            for f, s in zip(feature_names, shap_values[idx])
        ],
        'generated_at': datetime.now().isoformat()
    }
    
    return explanation


# Export sample explanations
sample_explanations = []
for i in range(min(10, len(X_test_reset))):
    exp = export_individual_explanation(
        i, xgb_shap_values, X_test_reset, feature_cols,
        xgb_explainer, xgb_pred, xgb_prob
    )
    sample_explanations.append(exp)

with open('sample_explanations.json', 'w') as f:
    json.dump(sample_explanations, f, indent=2)

print(f"\nExported {len(sample_explanations)} sample explanations to sample_explanations.json")

## 11. Summary

### Key Results

**SHAP Analysis:**
- Computed SHAP values for XGBoost and LightGBM models
- Generated summary plots showing feature importance
- Created force plots for individual prediction explanations

**Segment Performance:**
- Analyzed model performance across regions, user cohorts, and channels
- Identified potential performance disparities

**Fairness Checks:**
- Computed demographic parity (selection rate equality)
- Computed equalized odds (TPR/FPR equality)
- Generated fairness reports per sensitive attribute

### Artifacts Generated
- `shap_summary_xgboost.png` - XGBoost SHAP summary
- `shap_summary_lightgbm.png` - LightGBM SHAP summary
- `shap_bar_comparison.png` - Feature importance comparison
- `shap_force_plot_single.png` - Single prediction force plot
- `shap_dependence_total_revenue.png` - Dependence plot
- `segment_performance_region.png` - Regional performance
- `segment_performance_cohort.png` - Cohort performance
- `fairness_analysis.png` - Fairness visualization
- `shap_summary.json` - API-ready SHAP summary
- `sample_explanations.json` - Sample individual explanations

In [ ]:
print(f"\nNotebook execution complete at: {datetime.now().isoformat()}")
print("\nGenerated artifacts:")
for f in OUTPUT_DIR.glob('*.png'):
    print(f"  - {f.name}")
for f in OUTPUT_DIR.glob('*.json'):
    print(f"  - {f.name}")